In [90]:
from datetime import datetime
import calendar
import pprint
import pandas as pd
import numpy as np
import sys
import os

def notebook_settings():
    global control_sheet_file
    global report_group_file
    global invoice_2
    global invoice_5
    global mapping_file
    global faf_fee
    global royalty_fee
    
    os.chdir('/Users/rginsberg/Desktop/subwayexportconsolidatescripts/Data Files')
    
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_colwidth', 0)
    
    control_sheet_file = 'AccountingSoftwareControlSheetExport.xlsx'
    report_group_file = 'AccountingReportGroup.xlsx'
    invoice_2 = 'Invoice Statement Lisa Stores.xlsx'
    invoice_5 = 'Invoice Statement Steve Stores.xlsx'
    mapping_file = 'Master Mapping File-V2.xlsx'
    
    faf_fee = .045
    royalty_fee = .08

def import_columns(file):
    df = pd.read_excel(mapping_file)
    df = df[(df['File'] == file) & (df['is_import_column'] == 'Yes')]
    df_list = df['Column Name'].str.lower().tolist()
    return df_list

def neg_columns(file):
    df = pd.read_excel(mapping_file)
    df = df[(df['File'] == file) & (df['is_negative'] == 'Yes')]
    df_list = df['Rename To'].str.lower().tolist()
    return df_list

def daily_export_columns(file):
    df = pd.read_excel(mapping_file)
    df = df[(df['File'] == file) & (df['is_daily_export_column'] == 'Yes')]
    df_list = df['Rename To'].str.lower().tolist()
    return df_list

def weekly_export_columns(file):
    df = pd.read_excel(mapping_file)
    df = df[(df['File'] == file) & (df['is_weekly_export_column'] == 'Yes')]
    df_list = df['Rename To'].str.lower().tolist()
    return df_list

def ipc_export_columns(file):
    df = pd.read_excel(mapping_file)
    df = df[(df['File'] == file) & (df['is_ipc_export_column'] == 'Yes')]
    df_list = df['Rename To'].str.lower().tolist()
    return df_list


def rename_columns(file):
    df = pd.read_excel(mapping_file)
    df = df[(df['File'] == file) & (df['Rename To'].notnull()) ]
    df_dict = pd.Series(df['Rename To'].values,index=df['Column Name']).to_dict()
    return df_dict

def account_numbers():
    df = pd.read_excel(mapping_file, sheet_name=1)
    df_dict = pd.Series(df['Account #'].values,index=df['Memo']).to_dict()
    return df_dict

def column_check(df):
    
    if 'a/r' in df.columns:
        pass
    else:
        df['a/r'] = 0
        
    if 'cc-df130' in df.columns:
        pass
    else:
        df['cc-df130'] = 0
        
    if 'paypal' in df.columns:
        pass
    else:
        df['paypal'] = 0 
    
    if 'del-doordash' in df.columns:
        pass
    else:
        df['del-doordash'] = 0 
    
    if 'del-grubhub' in df.columns:
        pass
    else:
        df['del-grubhub'] = 0 
        
    if 'del-postmates' in df.columns:
        pass
    else:
        df['del-postmates'] = 0
        
    if 'del-ubereats' in df.columns:
        pass
    else:
        df['del-ubereats'] = 0
        
    if 'grubhub' in df.columns:
        pass
    else:
        df['grubhub'] = 0
        
    return df


def read_report_group():    
    #Read Report Group File
    df = pd.read_excel(report_group_file,
                       header = 7,
                       index_col = 'Date',
                       parse_dates=True).dropna(axis = 1, how='all')
    
    #Reset Index
    df = df.reset_index()
    
    #Convert Column Headers to Lower Case
    df.columns = df.columns.str.lower()
    
    #Check for Required Columns & Create if Missing
    df = column_check(df)
    
    #Drop Rows that contain the word 'Total'
    drop_df = df[df['store'].str.contains('Total|total')]
    df = df.drop(drop_df.index).reset_index()
    
    return df
    

def calculated_columns(df):
    
    #Create Calculated Columns for Report Group DF

    df['batch import'] = 0
    df['gift card net'] = df['cashcard'] - df['cash card sales']
    df['royalty fee'] =  df['adjusted net proceeds'] * royalty_fee
    df['faf/ad fee'] =  df['adjusted net proceeds'] * faf_fee
    df['vmd cc'] = df['visa'] +  df['mastercard'] + df['discover']
    
    df['doordash'] = df['doordash'] + df['del-doordash']
    df['postmates'] = df['postmates'] + df['del-postmates']
    df['uber'] = df['uber'] + df['del-ubereats']
    df['grubhub'] = df['grubhub'] + df['del-grubhub']
    
    df['doordash-sales'] = (df['doordash'])
    df['postmates-sales'] = (df['postmates'])
    df['uber-sales'] = (df['uber'] )
    df['grubhub-sales'] = (df['grubhub'])
    
    df['daily delivery sales offset'] = (df['third party prepaid'] + 
                                         df['del-doordash'] +
                                         df['del-postmates'] +
                                         df['del-ubereats'] +
                                         df['del-grubhub'])
    
    df['commission'] = df['cash card sales'] * .025
    df['cash card sales commission'] = df['cash card sales'] * .025
    df['cash card redeemed fee'] = df['cashcard'] * .025
    df['paypal fee 2%'] = df['paypal'] * .02
    df['7% call center'] = df['catering center'] * .07
    df['fees'] = ((df['catering center'] * .07) +
                  (df['cashcard'] * .025) +
                  (df['paypal'] * .02))
    
    df['net ipc'] = (df['catering center'] +
                     df['cashcard'] -
                     df['cash card sales'] +
                     df['commission'] -
                     df['fees'] +
                     df['paypal'])

    return df

def report_group_daily_df():
    df = calculated_columns(read_report_group())
    
    #Rename Columns
    df = df.rename(columns = rename_columns('Report Group'))
    
    #Multiply columns by specific columns by -1
    df[neg_columns('Report Group')] = df[neg_columns('Report Group')] * -1
    
    df = df[daily_export_columns('Report Group')]
    return df


def control_sheet_daily_df():
    #Read Control Sheet File
    control_sheet_df = pd.read_excel(control_sheet_file,
                                     header = 4,
                                     index_col = 'Date',
                                     parse_dates = True).dropna(axis = 1, how='all').reset_index()
    
    #Convert Column Headers to Lower Case
    control_sheet_df.columns = control_sheet_df.columns.str.lower()

    #Rename Columns
    control_sheet_df = control_sheet_df.rename(columns = rename_columns('Control Sheet'))
    
    #Multiply columns by specific columns by -1
    control_sheet_df[neg_columns('Control Sheet')] = control_sheet_df[neg_columns('Control Sheet')] * -1
    
    #Select Only Required Columns
    control_sheet_df = control_sheet_df[daily_export_columns('Control Sheet')]
    
    control_sheet_df['batch import'] = 0
    
    return control_sheet_df

def process_files(df):
    notebook_settings()
    global result

    
    result = pd.DataFrame(columns= {'TRNSID!',
                                    'Date',
                                    'Memo',
                                    'Amount',
                                    'Class',
                                    'Account #'
                                   })
    count = 0
    for col in df.columns:
        count=0
        for i in df[col]:
            transid = "Combo-" + str(df['store'][count])
            classname = str(df['store'][count])
            memo = col + " " + str(df['store'][count])
            accountname = col + " " + str(df['store'][count])
            amount = i
            date = df['date'][count]
        
            data = {'TRNSID!': transid,
                    'Memo': memo,
                    'Class': classname,
                    'Amount': amount,
                    'Date': date,
                    'Account #': accountname
               }

            result = result.append(data, ignore_index=True)
            count += 1
        count += 1
    
    result.loc[(result['Memo'].str.contains('batch import')), '!TRNS'] = 'Yes'
        
    result = result.drop(result.loc[result['Memo'].str.contains('Store|Date|Received|store|date|received|index')].index)
    result['Amount'] = pd.to_numeric(result['Amount'], errors='coerce').round(2)
    result = result.sort_values(by=['Class', 'Date', '!TRNS'])
    
    #Account # Mapping
    result['Account #'] = result['Account #'].map(account_numbers()).fillna(0).astype(int)
    
    return result


def read_invoice(file):
    df = pd.read_excel(file,
                       header = None, 
                       skiprows = 6,
                      ).dropna(axis = 1, how='all').dropna(how='all')
    
    df = df.rename(columns={0: 0,
                            1: 1,
                            2: 'item',
                            3: 3,
                            4: 4,
                            5: 'date',
                            6: 'memo',
                            7: 7,
                            8: 8,
                            9: 9,
                            10: 10,
                            13: 13,
                            17: 'amount',
                            19: 'store',
                            20: 20}
                  )
    
    df['category'] = df['item']
    
    for i in df[df['store'] == 'Karen Fisher'].index:
        df.at[i, 'store'] = np.NaN
        df.at[i + 1, 'store'] = np.NaN
        
    df['date'] = df['date'].fillna(method='ffill')
    df['store'] = df['store'].fillna(method='ffill')
    df['category'] = df['category'].str.replace('Item', '').fillna(np.NaN)
    df = df.drop(df.loc[df['item'].str.contains('Item', na=False)].index)
    df['category'] = df['category'].fillna(method='ffill')
    df['category'] = df['category'].str.strip().str.lower()
    df['memo'] = (df['memo'] + "-Invoice #" + df['item'].astype(str) + "-" + df['category']).str.lower()
    
    conditions = [df['memo'].str.startswith('ad waiver - delivery - postmates', na=False),
                  df['memo'].str.startswith('ad waiver - delivery - uber', na=False),
                  df['memo'].str.startswith('ad waiver - delivery - grubhub', na=False),
                  df['memo'].str.startswith('ad waiver - delivery - doordash', na=False),
                  df['memo'].str.startswith('loyalty surprise reward reimbursement', na=False),
                  df['memo'].str.startswith('loyalty sales tax reimbursement', na=False),
                  df['memo'].str.startswith('loyalty reward reimbursement', na=False),
                  df['memo'].str.startswith('royalty waiver - delivery - postmates', na=False),
                  df['memo'].str.startswith('royalty waiver - delivery - uber', na=False),
                  df['memo'].str.startswith('royalty waiver - delivery - grubhub', na=False),
                  df['memo'].str.startswith('royalty waiver - delivery - doordash', na=False),
                  (df['memo'].str.contains('salescorrection', na=False) & df['memo'].str.contains('advertising', na=False)),
                  (df['memo'].str.contains('salescorrection', na=False) & df['memo'].str.contains('royalty', na=False))
                 ]

    choices = ['7406', #AD Waiver - Delivery - Postmates
               '7406', # AD Waiver - Delivery - UberEats
               '7406', # AD Waiver - Delivery - Grubhub
               '7406', # AD Waiver - Delivery - Doordash
               '5407', # Loyalty Surprise Reward Reimbursement
               '2206', # Loyalty Sales Tax Reimbursement
               '5407', # Loyalty Reward Reimbursement
               '7301', # Royalty Waiver - Delivery - Postmates
               '7301', # Royalty Waiver - Delivery - Ubereats
               '7301', # Royalty Waiver - Delivery - Grubhub
               '7301', # Royalty Waiver - Delivery - Doordash
               '7405', #Sales Correction Advertising Account
               '7300', #Sales Correction Royalty Account
          ]
    
    df['account #'] = np.select(conditions, choices, default='Combo Purchases')
    
    
    df = df[['date',
             'memo',
             'amount',
             'store',
             'account #',
             'category']].dropna()
    
    return df

def process_invoices():
    df = read_invoice(invoice_5).append(read_invoice(invoice_2)).reset_index(drop=True)
    
    result = pd.DataFrame(columns= {'Date',
                                    'Memo',
                                    'Amount',
                                    'Class',
                                    'Account #'
                                   })
    for i in df.index:
        transid = "Combo-" + str(df['store'][i])
        date = df['date'][i]
        classname = str(df['store'][i])
        memo = df['memo'][i]
        amount = df['amount'][i]
        account_number = df['account #'][i]
        
        data = {'TRNSID!': transid,
                'Memo': memo,
                'Amount': amount,
                'Date': date,
                'Class': classname,
                'Account #': account_number
               }
                
        result = result.append(data, ignore_index=True)
        
    return result

def get_royalty_faf_df():
    df = read_report_group()
    
    #Get W/E Date
    we_date = df['date'][df.index[-1]]

    #Calculate Columns
    df['royalty fee'] =  df['adjusted net proceeds'] * royalty_fee
    df['faf/ad fee'] =  df['adjusted net proceeds'] * faf_fee
    
    
    #Create Weekly DF
    df = df.groupby('store').sum().round(2).reset_index()
    df['date'] = we_date
    

    df = df[weekly_export_columns('Report Group')]
    df['store'] = pd.to_numeric(df['store'], errors='ignore')
    
    return df

def get_invoice_df():
    df = read_invoice(invoice_5).append(read_invoice(invoice_2))
    df = df.groupby(['store', 'category']).sum().reset_index()
    df = df.pivot(index='store', columns='category', values='amount')
    df = df.reset_index()
    return df

def process_royalty_faf():
    df = get_royalty_faf_df().merge(get_invoice_df(), how= 'left', on = 'store')
    
    df['cash - faf'] = (df['faf/ad fee'] + df['advertising']) * -1
    df['cash - roy'] = (df['royalty fee'] + df['royalty']) * -1
    df['batch import'] = 0
    
    df = df[['store', 'date','faf/ad fee', 'royalty fee','cash - faf', 'cash - roy', 'batch import' ]]
    
    df = process_files(df).append(process_invoices())
    
    df = df.sort_values(by=['Class', 'Date', '!TRNS', 'Memo']).reset_index(drop=True)
    return df

def process_ipc():
    df = read_report_group()
    df = calculated_columns(df)
    df = df.rename(columns = rename_columns('Report Group'))
    df[neg_columns('Report Group')] = df[neg_columns('Report Group')] * -1
    df['paypal'] = df['paypal'] * -1
    df = df[ipc_export_columns('Report Group')]
    we_date = df['date'][df.index[-1]]
    df = df.groupby('store').sum().round(2).reset_index()
    df['date'] = we_date
    return df


def get_final_result():
    result = process_files(report_group_daily_df()).append([process_files(control_sheet_daily_df()),
                                                            process_royalty_faf(),
                                                            process_files(process_ipc())])
    return result

def create_excel():
    date = str(datetime.now())
    file_name = '/Users/rginsberg/Downloads/Tryan__Export_v_1.0' + date + '.xlsx'
    with pd.ExcelWriter(file_name, datetime_format='mm yyyy',) as writer:
        get_final_result().to_excel(writer, sheet_name='Export File')
    return ('Thank You. Your File Has Been Processed')

notebook_settings()
create_excel()
#get_final_result().groupby('Class').sum().round(2)
#account_numbers()



'Thank You. Your File Has Been Processed'